## 1. scraping one property using selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager

# URL to scrape
urls = [
    "https://immovlan.be/en/detail/villa/for-sale/9051/sint-denijs-westrem/rbu55821",
]

# Firefox options (headless)
options = Options()
options.add_argument("--headless")

# Loop through URLs
for url in urls:
    driver = None
    try:
        # Launch Firefox
        service = Service(GeckoDriverManager().install())
        driver = webdriver.Firefox(service=service, options=options)
        driver.get(url)

        # -----------------------
        # Extract property ID
        # -----------------------
        try:
            property_id = url.split('/')[-1]
        except:
            property_id = None

        # -----------------------
        # Extract postal code & locality
        # -----------------------
        try:
            city_tag = driver.find_element(By.CSS_SELECTOR, ".city-line")
            city_text = city_tag.text.strip()
            postal_code = city_text.split()[0]
            locality_name = " ".join(city_text.split()[1:])
        except:
            postal_code = None
            locality_name = None

        # -----------------------
        # Extract price
        # -----------------------
        try:
            price_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_price_data")
            price = price_tag.text.strip()
        except:
            price = None

        # -----------------------
        # Extract property type & subtype
        # -----------------------
        try:
            type_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_title_main")
            type_words = type_tag.text.strip().split()
            property_type = type_words[0]
            subtype = " ".join(type_words[1:]) if len(type_words) > 1 else ""
        except:
            property_type = None
            subtype = None
            

        # -----------------------
        # Print results
        # -----------------------
        print("Property ID:", property_id)
        print("Postal Code:", postal_code)
        print("Locality:", locality_name)
        print("Price:", price)
        print("Type:", property_type)
        print("Subtype:", subtype)
        print("-" * 40)

    except:
        print("Error loading page:", url)
    finally:
        if driver:
            driver.quit()

Property ID: rbu55821
Postal Code: 9051
Locality: Sint-Denijs-Westrem
Price: 860 000 €
Type: Villa
Subtype: for sale - Sint-Denijs-Westrem RBU55821
----------------------------------------


## 2. A property scraping function using selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait    # added
from selenium.webdriver.support import expected_conditions as EC #added
import csv  # <-- added for saving results


def scrape_properties(urls, output_file="properties.csv"):  # <-- added function definition
    # Firefox options (headless)
    options = Options()
    options.add_argument("--headless")

    # Prepare CSV file
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["URL", "Terrace", "Garden", "Garden area (m²)", "Property ID", "Postal Code", "Locality", "Price", "Type"])

        # Loop through URLs
        for url in urls:
            driver = None
            try:
                # Launch Firefox
                service = Service(GeckoDriverManager().install())
                driver = webdriver.Firefox(service=service, options=options)
                driver.get(url)

                # ---- Handle cookie consent automatically ----
                try:
                    # Wait for the "Agree and close" button to be clickable
                    accept_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
                    )
                    accept_button.click()
                    print("Cookies accepted.")
                except Exception as e:
                    print("No cookie popup found or could not click the accept button:", e)
                # ---- End cookie handling ----

                try:
                    terrace_tag = driver.find_element(By.CSS_SELECTOR, "h4:contains('Terrace') + p")
                    terrace_text = terrace_tag.text.strip()
                    terrace = 1 if "yes" in terrace_text.lower() else 0
                except:
                    terrace = None 

                # GARDEN: 1 = Yes, 0 = No
                try:
                    garden_blocks = driver.find_elements(By.CSS_SELECTOR, "div")
                    garden = 0  # default
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "garden" == h4_text:  # exact match for Garden
                                garden_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden = 1 if "yes" in garden_text.lower() else 0
                                break
                        except:
                            continue
                except:
                    garden = 0

                # GARDEN AREA (m²): integer or None
                try:
                    garden_area = None
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "surface garden" in h4_text:
                                garden_area_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden_area = int(garden_area_text.replace("m²", "").replace("m", "").strip())
                                break
                        except:
                            continue
                except:
                    garden_area = None

                # -----------------------
                # Extract property ID
                # -----------------------
                try:
                    property_id = url.split('/')[-1]
                except:
                    property_id = None

                # -----------------------
                # Extract postal code & locality
                # -----------------------
                try:
                    city_tag = driver.find_element(By.CSS_SELECTOR, ".city-line")
                    city_text = city_tag.text.strip()
                    postal_code = city_text.split()[0]
                    locality_name = " ".join(city_text.split()[1:])
                except:
                    postal_code = None
                    locality_name = None

                # -----------------------
                # Extract price
                # -----------------------
                try:
                    price_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_price_data")
                    price = price_tag.text.strip()
                except:
                    price = None

                # -----------------------
                # Extract property type & subtype
                # -----------------------
                try:
                    type_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_title_main")
                    type_words = type_tag.text.strip().split()
                    property_type = type_words[0]
                except:
                    property_type = None

                # -----------------------
                # Print results
                # -----------------------
                print("Terrace:", terrace)
                print("Garden:", garden)
                print("Garden area (m²):", garden_area)
                print("Property ID:", property_id)
                print("Postal Code:", postal_code)
                print("Locality:", locality_name)
                print("Price:", price)
                print("Type:", property_type)
                print("-" * 40)

                # Write row to CSV file
                writer.writerow([url, terrace, garden, garden_area, property_id, postal_code, locality_name, price, property_type])

            except:
                print("Error loading page:", url)
            finally:
                if driver:
                    driver.quit()


# Example use:
urls = [
    "https://immovlan.be/en/detail/villa/for-sale/9051/sint-denijs-westrem/rbu55821", "https://immovlan.be/en/detail/residence/for-sale/4900/spa/vbd48417","https://immovlan.be/en/detail/apartment/for-sale/9000/gent/rbu59338"
]
scrape_properties(urls)  # <-- added function call


Cookies accepted.
Terrace: None
Garden: 1
Garden area (m²): 470
Property ID: rbu55821
Postal Code: 9051
Locality: Sint-Denijs-Westrem
Price: 860 000 €
Type: Villa
----------------------------------------
Cookies accepted.
Terrace: None
Garden: 1
Garden area (m²): 472
Property ID: vbd48417
Postal Code: 4900
Locality: Spa
Price: 199 000 €
Type: Residence
----------------------------------------
Cookies accepted.
Terrace: None
Garden: 0
Garden area (m²): None
Property ID: rbu59338
Postal Code: 9000
Locality: Gent
Price: 325 000 €
Type: Apartment
----------------------------------------


## 3. Collect apartment URLs, including new construction apartments, based on keywords.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from webdriver_manager.firefox import GeckoDriverManager
import csv
import time

# -----------------------
# Handle cookies popup
# -----------------------
def handle_cookies(driver, timeout=10):
    selectors = [
        "#onetrust-accept-btn-handler",
        "#didomi-notice-agree-button",
    ]
    for selector in selectors:
        try:
            cookie_btn = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
            )
            try:
                cookie_btn.click()
                print(f"Cookies accepted ({selector})")
                return
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].scrollIntoView(true);", cookie_btn)
                time.sleep(0.5)
                cookie_btn.click()
                print(f"Cookies accepted after scrolling ({selector})")
                return
        except TimeoutException:
            continue
    print("No cookie popup found.")

# -----------------------
# Collect apartment URLs with precise filter
# -----------------------
def collect_apartment_urls(search_url, output_file="apartment_urls.csv"):
    """
    Collect apartment URLs, including new construction apartments, based on keywords.
    """
    options = Options()
    options.add_argument("--headless")  
    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)
    all_urls = set()

    # Keywords for apartment-related listings
    apartment_keywords = [
        "/apartment/",
        "groundfloor",
        "residence",
        "penthouse",
        "studio",
        "duplex",
        "loft",
        "triplex",
        "isnewconstruction=only",
        "/projectdetail/"
    ]

    try:
        driver.get(search_url)
        handle_cookies(driver)
        time.sleep(2)

        while True:
            print("Scraping current page for apartment URLs...")

            # Wait for property cards
            try:
                WebDriverWait(driver, 20).until(
                    EC.presence_of_all_elements_located(
                        (By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
                    )
                )
            except TimeoutException:
                print("No property links found on this page. Skipping...")
                continue

            # Scroll for lazy-loading
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            # Collect apartment links
            links = driver.find_elements(By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
            for link in links:
                href = link.get_attribute("href")
                if href and any(keyword in href.lower() for keyword in apartment_keywords):
                    all_urls.add(href.split("?")[0])

            print(f"Found {len(links)} links on this page (total apartments so far: {len(all_urls)})")

            # Go to next page if available
            try:
                next_button = driver.find_element(
                    By.XPATH, "//a[contains(@class,'pagination__next') and not(contains(@class,'disabled'))]"
                )
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(1)
                next_button.click()
                time.sleep(3)
            except:
                print("Reached the last page.")
                break

    finally:
        driver.quit()

    # Save URLs to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["URL"])
        for u in sorted(all_urls):
            writer.writerow([u])

    print(f"Saved {len(all_urls)} unique apartment URLs to '{output_file}'")
    return list(sorted(all_urls))


# -----------------------
# Example usage
# -----------------------
if __name__ == "__main__":
    search_url = (
        "https://immovlan.be/en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=apartment&propertysubtypes=apartment,penthouse,studio,ground-floor,duplex,loft,triplex&towns=4900-spa&noindex=1"
    )
    apartment_urls = collect_apartment_urls(search_url)
    print(f"Total collected apartment URLs: {len(apartment_urls)}")
    for u in apartment_urls[:]:
        print(u)


No cookie popup found.
Scraping current page for apartment URLs...
Found 20 links on this page (total apartments so far: 20)
Reached the last page.
Saved 20 unique apartment URLs to 'apartment_urls.csv'
Total collected apartment URLs: 20
https://immovlan.be/en/detail/apartment/for-sale/4900/spa/vbb82041
https://immovlan.be/en/detail/apartment/for-sale/4900/spa/vbc15133
https://immovlan.be/en/detail/apartment/for-sale/4900/spa/vbc15134
https://immovlan.be/en/detail/apartment/for-sale/4900/spa/vbc15136
https://immovlan.be/en/detail/apartment/for-sale/4900/spa/vbc15137
https://immovlan.be/en/detail/apartment/for-sale/4900/spa/vbc15150
https://immovlan.be/en/detail/apartment/for-sale/4900/spa/vbc15158
https://immovlan.be/en/detail/apartment/for-sale/4900/spa/vbc53316
https://immovlan.be/en/detail/apartment/for-sale/4900/spa/vbc63045
https://immovlan.be/en/detail/apartment/for-sale/4900/spa/vbc97369
https://immovlan.be/en/detail/apartment/for-sale/4900/spa/vbd26412
https://immovlan.be/en/de

## 4. collecting URL of 5 pages

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, WebDriverException
from webdriver_manager.firefox import GeckoDriverManager
import csv
import time

# -----------------------
# Handle cookies popup
# -----------------------
def handle_cookies(driver, timeout=10):
    selectors = [
        "#onetrust-accept-btn-handler",
        "#didomi-notice-agree-button",
        "button[aria-label='Accept all']",
    ]
    for selector in selectors:
        try:
            cookie_btn = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
            )
            try:
                cookie_btn.click()
                print(f"Cookies accepted ({selector})")
                return
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].scrollIntoView(true);", cookie_btn)
                time.sleep(0.5)
                cookie_btn.click()
                print(f"Cookies accepted after scrolling ({selector})")
                return
        except TimeoutException:
            continue
    print("No cookie popup found.")

# -----------------------
# Collect property URLs (1 page)
# -----------------------
def collect_property_urls(full_url, output_file="property_urls.csv"):
    """
    Collect apartment and house URLs from a single Immovlan page.
    Filters by subtypes and ensures listings are 'for sale'.
    """
    # Keywords for filtering
    apartment_keywords = [
        "/apartment/",
        "apartment",
        "studio",
        "flat",
        "ground floor",
        "ground-floor",
        "duplex",
        "penthouse",
        "loft",
        "triplex",
        "new construction",
        "new-construction",
        "isnewconstruction=only",
        "/projectdetail/",

    ]
    house_keywords = [
        "/house/",
        "villa",
        "residence",
        "mixed building",
        "master house",
        "cottage",
        "bungalow",
        "chalet",
        "mansion",
        
    ]

    options = Options()
    options.add_argument("--headless")  # 

    options.add_argument("--width=1920")
    options.add_argument("--height=1080")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    service = Service(GeckoDriverManager().install())
    apartments = set()
    houses = set()

    try:
        driver = webdriver.Firefox(service=service, options=options)
        driver.get(full_url)
        handle_cookies(driver)
        time.sleep(2)

        print("Scraping current page for property URLs...")

        # Wait for property cards
        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
                )
            )
        except TimeoutException:
            print("No property links found on this page.")
            return [], []

        # Scroll to trigger lazy-loading
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # Collect property links
        links = driver.find_elements(By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
        for link in links:
            href = link.get_attribute("href")
            if href and "for-sale" in href.lower():
                if any(k.lower() in href.lower() for k in apartment_keywords):
                    apartments.add(href.split("?")[0])
                elif any(k.lower() in href.lower() for k in house_keywords):
                    houses.add(href.split("?")[0])

        print(f"Found {len(apartments)} apartment links and {len(houses)} house links on this page.")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")
    finally:
        driver.quit()

    # Save URLs to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Type", "URL"])
        for u in sorted(apartments):
            writer.writerow(["Apartment", u])
        for u in sorted(houses):
            writer.writerow(["House", u])

    print(f"Saved property URLs to '{output_file}'")
    print("\nList of collected URLs:")
    for u in sorted(apartments):
        print(f"Apartment: {u}")
    for u in sorted(houses):
        print(f"House: {u}")

    return list(sorted(apartments)), list(sorted(houses))


# -----------------------
# Example usage
# -----------------------
if __name__ == "__main__":
    # Single full URL
    full_url = (
        "https://immovlan.be/en/real-estate?transactiontypes=for-sale&propertytypes=house,apartment&provinces=liege&noindex=5"
    )

    apartments, houses = collect_property_urls(full_url)
    print(f"\nTotal apartments: {len(apartments)}")
    print(f"Total houses: {len(houses)}")


No cookie popup found.
Scraping current page for property URLs...
Found 1 apartment links and 4 house links on this page.
Saved property URLs to 'property_urls.csv'

List of collected URLs:
Apartment: https://immovlan.be/en/detail/apartment/for-sale/4053/embourg/vbd44032
House: https://immovlan.be/en/detail/residence/for-sale/4030/grivegnee/vbd32524
House: https://immovlan.be/en/detail/residence/for-sale/4190/xhoris/vbd45833
House: https://immovlan.be/en/detail/residence/for-sale/4219/wasseiges/vbd44721
House: https://immovlan.be/en/detail/residence/for-sale/4780/recht/vwd15127

Total apartments: 1
Total houses: 4


## 10 pages using multithread

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import threading
import csv
import time

# -----------------------
# Handle cookies popup
# -----------------------
def handle_cookies(driver, timeout=10):
    selectors = [
        "#onetrust-accept-btn-handler",
        "#didomi-notice-agree-button",
        "button[aria-label='Accept all']",
    ]
    for selector in selectors:
        try:
            cookie_btn = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
            )
            try:
                cookie_btn.click()
                print(f"Cookies accepted ({selector})")
                return
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].scrollIntoView(true);", cookie_btn)
                time.sleep(0.5)
                cookie_btn.click()
                print(f"Cookies accepted after scrolling ({selector})")
                return
        except TimeoutException:
            continue
    print("No cookie popup found.")

# -----------------------
# Scrape one page
# -----------------------
def scrape_page(page_url, apartments_set, houses_set, lock):
    apartment_keywords = [
        "/apartment/", "apartment", "studio", "flat",
        "ground floor", "ground-floor", "duplex", "penthouse",
        "loft", "triplex", "new construction", "new-construction",
        "isnewconstruction=only", "/projectdetail/"
    ]
    house_keywords = [
        "/house/", "villa", "residence", "mixed building",
        "master house", "cottage", "bungalow", "chalet", "mansion"
    ]

    options = Options()
    options.add_argument("--headless")  

    options.add_argument("--width=1920")
    options.add_argument("--height=1080")

    gecko_path = r"C:/Users/geckodriver.exe"
    service = Service(gecko_path)
    driver = webdriver.Firefox(service=service, options=options)

    try:
        driver.get(page_url)
        handle_cookies(driver)
        time.sleep(2)

        # Scroll to load all properties
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
                )
            )
        except TimeoutException:
            print(f"No property links found on {page_url}")
            return

        links = driver.find_elements(By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
        page_apartments = set()
        page_houses = set()
        for link in links:
            href = link.get_attribute("href")
            if href:
                href_clean = href.split("?")[0]
                if any(k.lower() in href_clean.lower() for k in apartment_keywords):
                    page_apartments.add(href_clean)
                elif any(k.lower() in href_clean.lower() for k in house_keywords):
                    page_houses.add(href_clean)

        # Safely update shared sets
        with lock:
            apartments_set.update(page_apartments)
            houses_set.update(page_houses)
        print(f"Scraped {len(page_apartments)} apartments and {len(page_houses)} houses from {page_url}")

    finally:
        driver.quit()

# -----------------------
# Main multi-threaded function
# -----------------------
def scrape_multiple_pages(base_url, num_pages=10, output_file="property_urls.csv"):
    apartments = set()
    houses = set()
    lock = threading.Lock()
    threads = []

    for page in range(1, num_pages + 1):
        page_url = f"{base_url}&page={page}"
        t = threading.Thread(target=scrape_page, args=(page_url, apartments, houses, lock))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    # Save to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Type", "URL"])
        for u in sorted(apartments):
            writer.writerow(["Apartment", u])
        for u in sorted(houses):
            writer.writerow(["House", u])

    print(f"\n✅ Finished scraping {num_pages} pages.")
    print(f"Total apartments: {len(apartments)}, Total houses: {len(houses)}")
    print(f"Saved to '{output_file}'")
    return list(sorted(apartments)), list(sorted(houses))

# -----------------------
# Example usage
# -----------------------
if __name__ == "__main__":
    base_url = (
        "https://immovlan.be/en/real-estate?"
        "transactiontypes=for-sale&"
        "propertytypes=house,apartment&"
        "provinces=liege&noindex=1"
    )
    apartments, houses = scrape_multiple_pages(base_url, num_pages=10)


Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
Scraped 5 apartments and 14 houses from https://immovlan.be/en/real-estate?transactiontypes=for-sale&propertytypes=house,apartment&provinces=liege&noindex=1&page=2
Scraped 16 apartments and 4 houses from https://immovlan.be/en/real-estate?transactiontypes=for-sale&propertytypes=house,apartment&provinces=liege&noindex=1&page=1
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
Scraped 7 apartments and 12 houses from https://immovlan.be/en/real-estate?transactiontypes=for-sale&propertytypes=house,apartment&provinces=liege&noindex=1&page=10
Scraped 5 apartments and 12 houses from https://immovlan.be/en/real-estate?transacti

## for each province

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import threading
import csv
import time

# -----------------------
# Handle cookie popups
# -----------------------
def handle_cookies(driver, timeout=10):
    selectors = [
        "#onetrust-accept-btn-handler",
        "#didomi-notice-agree-button",
        "button[aria-label='Accept all']",
    ]
    for selector in selectors:
        try:
            cookie_btn = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
            )
            try:
                cookie_btn.click()
                print(f"Cookies accepted ({selector})")
                return
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].scrollIntoView(true);", cookie_btn)
                time.sleep(0.5)
                cookie_btn.click()
                print(f"Cookies accepted after scrolling ({selector})")
                return
        except TimeoutException:
            continue
    print("No cookie popup found.")

# -----------------------
# Scrape a single province
# -----------------------
def scrape_province(province_url, province_name, apartments_set, houses_set, lock):
    apartment_keywords = [
        "/apartment/", "apartment", "studio", "flat",
        "ground floor", "ground-floor", "duplex", "penthouse",
        "loft", "triplex", "new construction", "new-construction",
        "isnewconstruction=only", "/projectdetail/"
    ]
    house_keywords = [
        "/house/", "villa", "residence", "mixed building",
        "master house", "cottage", "bungalow", "chalet", "mansion"
    ]

    options = Options()
    options.add_argument("--headless")  # ✅ correct

    options.add_argument("--width=1920")
    options.add_argument("--height=1080")

    gecko_path = r"C:/Users/geckodriver.exe"
    service = Service(gecko_path)
    driver = webdriver.Firefox(service=service, options=options)

    try:
        driver.get(province_url)
        handle_cookies(driver)
        time.sleep(2)

        print(f"🌍 Scraping {province_name}...")

        # Scroll to load lazy-loaded properties
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
                )
            )
        except TimeoutException:
            print(f"No property links found in {province_name}")
            return

        links = driver.find_elements(By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
        province_apartments = set()
        province_houses = set()
        for link in links:
            href = link.get_attribute("href")
            if href:
                href_clean = href.split("?")[0]
                if any(k.lower() in href_clean.lower() for k in apartment_keywords):
                    province_apartments.add(href_clean)
                elif any(k.lower() in href_clean.lower() for k in house_keywords):
                    province_houses.add(href_clean)

        with lock:
            apartments_set.update(province_apartments)
            houses_set.update(province_houses)

        print(f"✅ {province_name}: {len(province_apartments)} apartments, {len(province_houses)} houses.")

    finally:
        driver.quit()

# -----------------------
# Scrape all provinces
# -----------------------
def scrape_all_provinces():
    provinces = {
        "Flemish Brabant": "flemish-brabant",
        "Walloon Brabant": "walloon-brabant",
        "East Flanders": "east-flanders",
        "West Flanders": "west-flanders",
        "Hainaut": "hainaut",
        "Liège": "liege",
        "Limburg": "limburg",
        "Namur": "namur"
    }

    base_url = "https://immovlan.be/en/real-estate?transactiontypes=for-sale&propertytypes=house,apartment&provinces={prov}&noindex=1"
    apartments = set()
    houses = set()
    lock = threading.Lock()
    threads = []

    start_time = time.time()

    for province_name, province_slug in provinces.items():
        province_url = base_url.format(prov=province_slug)
        t = threading.Thread(target=scrape_province, args=(province_url, province_name, apartments, houses, lock))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    # Save results
    with open("belgium_property_urls.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Type", "URL"])
        for u in sorted(apartments):
            writer.writerow(["Apartment", u])
        for u in sorted(houses):
            writer.writerow(["House", u])

    end_time = time.time()
    print(f"\n🏁 Finished scraping in {end_time - start_time:.2f} seconds")
    print(f"Total apartments: {len(apartments)}, Total houses: {len(houses)}")
    print("Saved to belgium_property_urls.csv")

# -----------------------
# Run
# -----------------------
if __name__ == "__main__":
    scrape_all_provinces()


Cookies accepted (#didomi-notice-agree-button)
🌍 Scraping Liège...
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
✅ Liège: 16 apartments, 4 houses.
🌍 Scraping Namur...
🌍 Scraping Flemish Brabant...
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
🌍 Scraping Limburg...
✅ Namur: 6 apartments, 14 houses.
🌍 Scraping Walloon Brabant...
Cookies accepted (#didomi-notice-agree-button)
🌍 Scraping West Flanders...
Cookies accepted (#didomi-notice-agree-button)
✅ Limburg: 14 apartments, 6 houses.
🌍 Scraping East Flanders...
🌍 Scraping Hainaut...
✅ West Flanders: 12 apartments, 8 houses.
✅ East Flanders: 14 apartments, 5 houses.
✅ Hainaut: 5 apartments, 14 houses.
No property links found in Flemish Brabant
No property links found in Walloon Brabant

🏁 Finished scraping in 48.49 seconds
Total apartments: 67, Total houses: 51
Saved to belgium_property_urls.csv


## for first 10 pages of each province

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import threading
import csv
import time

# -----------------------
# Handle cookie popups
# -----------------------
def handle_cookies(driver, timeout=5):
    selectors = [
        "#onetrust-accept-btn-handler",
        "#didomi-notice-agree-button",
        "button[aria-label='Accept all']",
    ]
    for selector in selectors:
        try:
            cookie_btn = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
            )
            try:
                cookie_btn.click()
                print(f"Cookies accepted ({selector})")
                return
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].scrollIntoView(true);", cookie_btn)
                time.sleep(0.5)
                cookie_btn.click()
                print(f"Cookies accepted after scrolling ({selector})")
                return
        except TimeoutException:
            continue
    print("No cookie popup found.")

# -----------------------
# Scrape a single province (first 10 pages)
# -----------------------
def scrape_province(province_slug, province_name, apartments_set, houses_set, lock, max_pages=2):
    apartment_keywords = [
        "/apartment/", "apartment", "studio", "flat",
        "ground floor", "ground-floor", "duplex", "penthouse",
        "loft", "triplex", "new construction", "new-construction",
        "isnewconstruction=only", "/projectdetail/"
    ]
    house_keywords = [
        "/house/", "villa", "residence", "mixed building",
        "master house", "cottage", "bungalow", "chalet", "mansion"
    ]

    options = Options()
    options.add_argument("--headless")  # ✅ correct

    options.add_argument("--width=1920")
    options.add_argument("--height=1080")

    # Specify your GeckoDriver path here
    gecko_path = r"C:/Users/geckodriver.exe"
    service = Service(gecko_path)
    driver = webdriver.Firefox(service=service, options=options)

    try:
        for page in range(1, max_pages + 1):
            province_url = (
                f"https://immovlan.be/en/real-estate?"
                f"transactiontypes=for-sale&propertytypes=house,apartment&provinces={province_slug}&noindex=1&page={page}"
            )
            driver.get(province_url)
            handle_cookies(driver)
            time.sleep(2)

            print(f"🌍 {province_name} - Page {page}")

            # Scroll to load lazy-loaded properties
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located(
                        (By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
                    )
                )
            except TimeoutException:
                print(f"No property links found on page {page} in {province_name}")
                continue

            links = driver.find_elements(By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
            province_apartments = set()
            province_houses = set()
            for link in links:
                href = link.get_attribute("href")
                if href:
                    href_clean = href.split("?")[0]
                    if any(k.lower() in href_clean.lower() for k in apartment_keywords):
                        province_apartments.add(href_clean)
                    elif any(k.lower() in href_clean.lower() for k in house_keywords):
                        province_houses.add(href_clean)

            with lock:
                apartments_set.update(province_apartments)
                houses_set.update(province_houses)

            print(f"✅ Page {page} - {province_name}: {len(province_apartments)} apartments, {len(province_houses)} houses.")

    finally:
        driver.quit()

# -----------------------
# Scrape all provinces
# -----------------------
def scrape_all_provinces(max_pages=10):
    provinces = {
        "Flemish Brabant": "flemish-brabant",
        "Walloon Brabant": "walloon-brabant",
        "East Flanders": "east-flanders",
        "West Flanders": "west-flanders",
        "Hainaut": "hainaut",
        "Liège": "liege",
        "Limburg": "limburg",
        "Namur": "namur"
    }

    apartments = set()
    houses = set()
    lock = threading.Lock()
    threads = []

    start_time = time.time()

    for province_name, province_slug in provinces.items():
        t = threading.Thread(
            target=scrape_province,
            args=(province_slug, province_name, apartments, houses, lock, max_pages)
        )
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    # Save results to CSV
    with open("belgium_property_urls.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Type", "URL"])
        for u in sorted(apartments):
            writer.writerow(["Apartment", u])
        for u in sorted(houses):
            writer.writerow(["House", u])

    total_properties = len(apartments) + len(houses)
    end_time = time.time()
    print(f"\n🏁 Finished scraping in {end_time - start_time:.2f} seconds")
    print(f"Total apartments: {len(apartments)}, Total houses: {len(houses)}, Total properties: {total_properties}")
    print("Saved to belgium_property_urls.csv")

# -----------------------
# Run the scraper
# -----------------------
if __name__ == "__main__":
    scrape_all_provinces(max_pages=50)


Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
🌍 Limburg - Page 1
Cookies accepted (#didomi-notice-agree-button)
🌍 Walloon Brabant - Page 1
✅ Page 1 - Limburg: 14 apartments, 6 houses.
🌍 Hainaut - Page 1
Cookies accepted (#didomi-notice-agree-button)
🌍 Liège - Page 1
✅ Page 1 - Hainaut: 5 apartments, 14 houses.
Cookies accepted (#didomi-notice-agree-button)
✅ Page 1 - Liège: 16 apartments, 4 houses.
🌍 Namur - Page 1
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
🌍 East Flanders - Page 1
✅ Page 1 - Namur: 7 apartments, 13 houses.
🌍 West Flanders - Page 1
Cookies accepted (#didomi-notice-agree-button)
✅ Page 1 - East Flanders: 14 apartments, 5 houses.
🌍 Flemish Brabant - Page 1
✅ Page 1 - West Flanders: 12 apartments, 8 houses.
No property links found on page 1 in Walloon Brabant
No cookie popup found.
🌍 Limburg - Page 2
No cookie popup found.
✅ Page 2 - Limburg: 12 apartments, 7 houses.
🌍 Haina

## scraping URLS for both house and apartment together

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import threading
import csv
import time


# -----------------------
# Handle cookie popups
# -----------------------
def handle_cookies(driver, timeout=10):
    selectors = [
        "#onetrust-accept-btn-handler",
        "#didomi-notice-agree-button",
        "button[aria-label='Accept all']",
    ]
    for selector in selectors:
        try:
            cookie_btn = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
            )
            try:
                cookie_btn.click()
                print(f"Cookies accepted ({selector})")
                return
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].scrollIntoView(true);", cookie_btn)
                time.sleep(0.5)
                cookie_btn.click()
                print(f"Cookies accepted after scrolling ({selector})")
                return
        except TimeoutException:
            continue
    print("No cookie popup found.")


# -----------------------
# Scrape a single province (first N pages)
# -----------------------
def scrape_province(province_slug, province_name, properties_set, lock, max_pages=10):
    options = Options()
    options.add_argument("--headless")  # 
    options.add_argument("--width=1920")
    options.add_argument("--height=1080")

    # Path to your GeckoDriver
    gecko_path = r"C:/Users/geckodriver.exe"
    service = Service(gecko_path)
    driver = webdriver.Firefox(service=service, options=options)

    try:
        for page in range(1, max_pages + 1):
            province_url = (
                f"https://immovlan.be/en/real-estate?"
                f"transactiontypes=for-sale&propertytypes=house,apartment&provinces={province_slug}&noindex=1&page={page}"
            )
            driver.get(province_url)
            handle_cookies(driver)
            time.sleep(2)

            print(f"🌍 {province_name} - Page {page}")

            # Scroll down to load all listings
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Wait for property links
            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located(
                        (By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
                    )
                )
            except TimeoutException:
                print(f"No property links found on page {page} in {province_name}")
                continue

            # Extract links
            links = driver.find_elements(By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
            page_urls = {link.get_attribute("href").split("?")[0] for link in links if link.get_attribute("href")}

            # Thread-safe update
            with lock:
                properties_set.update(page_urls)

            print(f"✅ Page {page} - {province_name}: {len(page_urls)} properties found.")

    finally:
        driver.quit()


# -----------------------
# Scrape all provinces
# -----------------------
def scrape_all_provinces(max_pages=10):
    provinces = {
        "Flemish Brabant": "flemish-brabant",
        "Walloon Brabant": "walloon-brabant",
        "East Flanders": "east-flanders",
        "West Flanders": "west-flanders",
        "Hainaut": "hainaut",
        "Liège": "liege",
        "Limburg": "limburg",
        "Namur": "namur",
        "Antwerpen": "antwerpen"
    }

    properties = set()
    lock = threading.Lock()
    threads = []

    start_time = time.time()

    # Start one thread per province
    for province_name, province_slug in provinces.items():
        t = threading.Thread(
            target=scrape_province,
            args=(province_slug, province_name, properties, lock, max_pages)
        )
        threads.append(t)
        t.start()

    # Wait for all threads to complete
    for t in threads:
        t.join()

    # Save to CSV
    with open("final_urls.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["URL"])
        for url in sorted(properties):
            writer.writerow([url])

    total_properties = len(properties)
    end_time = time.time()
    print(f"\n🏁 Finished scraping in {end_time - start_time:.2f} seconds")
    print(f"Total properties: {total_properties}")
    print("Saved to final_urls.csv")


# -----------------------
# Run
# -----------------------
if __name__ == "__main__":
    scrape_all_provinces(max_pages=100)


Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
🌍 West Flanders - Page 1
🌍 Flemish Brabant - Page 1
🌍 Walloon Brabant - Page 1
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
✅ Page 1 - West Flanders: 20 properties found.
🌍 Hainaut - Page 1
🌍 Namur - Page 1
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
Cookies accepted (#didomi-notice-agree-button)
✅ Page 1 - Hainaut: 20 properties found.
🌍 East Flanders - Page 1
✅ Page 1 - Namur: 20 properties found.
🌍 Liège - Page 1
🌍 Limburg - Page 1
✅ Page 1 - East Flanders: 20 properties found.
✅ Page 1 - Liège: 20 properties found.
✅ Page 1 - Limburg: 20 properties found.
No property links found on page 1 in Flemish Brabant
No property links found on page 1 in Walloon Brabant
No cookie popup found.
🌍 West Flanders - Page 2
No cookie popup found.
✅ Page 2 - West F

Exception in thread Thread-9 (scrape_province):
Traceback (most recent call last):
  File "c:\Users\welde\Desktop\immo-eliza-scraping\venv\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "c:\Users\welde\Desktop\immo-eliza-scraping\venv\Lib\site-packages\urllib3\connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
  File "C:\Users\welde\AppData\Local\Python\pythoncore-3.14-64\Lib\http\client.py", line 1430, in getresponse
    response.begin()
    ~~~~~~~~~~~~~~^^
  File "C:\Users\welde\AppData\Local\Python\pythoncore-3.14-64\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ~~~~~~~~~~~~~~~~~^^
  File "C:\Users\welde\AppData\Local\Python\pythoncore-3.14-64\Lib\http\client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^
  File "C:\Use

✅ Page 33 - West Flanders: 20 properties found.
No cookie popup found.
🌍 Hainaut - Page 33
✅ Page 33 - Hainaut: 20 properties found.
No cookie popup found.
No cookie popup found.
🌍 Flemish Brabant - Page 24
🌍 Walloon Brabant - Page 24
No cookie popup found.
🌍 Limburg - Page 33
✅ Page 33 - Limburg: 20 properties found.
No cookie popup found.
No cookie popup found.
🌍 Liège - Page 33
🌍 Namur - Page 33
✅ Page 33 - Liège: 20 properties found.
✅ Page 33 - Namur: 20 properties found.
No property links found on page 24 in Flemish Brabant
No property links found on page 24 in Walloon Brabant
No cookie popup found.
🌍 West Flanders - Page 34
✅ Page 34 - West Flanders: 20 properties found.
No cookie popup found.
🌍 Hainaut - Page 34
✅ Page 34 - Hainaut: 20 properties found.
No cookie popup found.
🌍 Limburg - Page 34
✅ Page 34 - Limburg: 20 properties found.
No cookie popup found.
No cookie popup found.
No cookie popup found.
No cookie popup found.
🌍 Liège - Page 34
🌍 Flemish Brabant - Page 25
🌍 Nam

## Function of property scraper from URL

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait    # added
from selenium.webdriver.support import expected_conditions as EC #added
import csv  # <-- added for saving results


def scrape_urls(urls, output_file="properties.csv"):  # <-- added function definition
    # Firefox options (headless)
    options = Options()
    options.add_argument("--headless")

    # Prepare CSV file
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow([" URL", " Property_id", " Property_type", " Locality_name", " Postal_code", " Build_year", " State" ," Furnished", " Number_rooms"," Equipped_kitchen", " Terrace"," Facades", " Garden", " garden_area"," swimming_pool", " price"])

        # Loop through URLs
        for url in urls:
            driver = None
            try:
                # Launch Firefox
                GECKO_PATH = r"C:\users\geckodriver.exe"  # <-- your local path
                service = Service(GECKO_PATH)
                driver = webdriver.Firefox(service=service, options=options)
                #print(f"\n Scraping {i} of {len(urls)} → {url}")
                driver.get(url)

                # ---- Handle cookie consent automatically ----
                
                try:
                    # Wait for the "Agree and close" button to be clickable
                    accept_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
                    )
                    accept_button.click()
                    print("Cookies accepted.")
                except Exception as e:
                    print("No cookie popup found or could not click the accept button:", e)
                # ---- End cookie handling ----

                
                ## TERRACE: 1 = Yes, 0 = No
                try:
                    terrace_tag = driver.find_element(By.CSS_SELECTOR, "h4:contains('Terrace') + p")
                    terrace_text = terrace_tag.text.strip()
                    terrace = 1 if "yes" in terrace_text.lower() else 0
                except:
                    terrace = None 

                # GARDEN: 1 = Yes, 0 = No
                try:
                    garden_blocks = driver.find_elements(By.CSS_SELECTOR, "div")
                    garden = 0  # default
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "garden" == h4_text:  # exact match for Garden
                                garden_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden = 1 if "yes" in garden_text.lower() else 0
                                break
                        except:
                            continue
                except:
                    garden = 0

                # GARDEN AREA (m²): integer or None
                try:
                    garden_area = None
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "surface garden" in h4_text:
                                garden_area_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden_area = int(garden_area_text.replace("m²", "").replace("m", "").strip())
                                break
                        except:
                            continue
                except:
                    garden_area = None
                try:
                    facades_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of facades')]/following-sibling::p").text
                    facades = int(facades_text)
                except:
                    facades = None


                    ################################  :-)
                # SWIMMING POOL: 1 = Yes, 0 = No
                try:
                    pool_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Swimming pool')]/following-sibling::p").text
                    swimming_pool = 1 if "Yes" in pool_text else 0
                except:
                    swimming_pool = 0


                ###########################"" ADDING MULTIPLE ITESM
                # STATE OF BUILDING: text or None
                try:
                    state = driver.find_element(By.XPATH, "//h4[contains(text(), 'State of the property')]/following-sibling::p").text
                except:
                    state = None

                # BUILD YEAR: integer or None 
                try:
                    build_year_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Build Year')]/following-sibling::p").text
                    build_year = int(build_year_text)
                except:
                    build_year = None

                # NUMBER OF ROOMS: integer or None
                try:
                    number_rooms_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of bedrooms')]/following-sibling::p").text
                    number_rooms = int(number_rooms_text)
                except:
                    number_rooms = None

                    # LIVING AREA (m²): integer or None
                try:
                    living_area_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Livable surface')]/following-sibling::p").text
                    living_area = int(living_area_text.replace("m²", "").replace("m", "").strip())
                except:
                    living_area = None

                # EQUIPPED KITCHEN: 1 = Yes, 0 = No, None if missing 
            
                try:
                    kitchen_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Kitchen equipment')]/following-sibling::p").text
                    if "equipped" in kitchen_text.lower():  # any text containing "equipped"
                        equipped_kitchen = 1
                    elif "not" in kitchen_text.lower():      # text containing "not"
                        equipped_kitchen = 0
                    else:
                        equipped_kitchen = None
                except:
                    equipped_kitchen = None

                # FURNISHED: 1 = Yes, 0 = No, None if missing
                try:
                    furnished_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Furnished')]/following-sibling::p").text
                    if "yes" in furnished_text.lower():
                        furnished = 1
                    elif "no" in furnished_text.lower():
                        furnished = 0
                    else:
                        furnished = None
                except:
                        furnished = None

                ###########################"" MULTIPLE ITEMS ADDED ABOVE  
                
                

                # -----------------------
                # Extract property ID
                # -----------------------
                try:
                    property_id = url.split('/')[-1]
                except:
                    property_id = None

                # -----------------------
                # Extract postal code & locality
                # -----------------------
                try:
                    city_tag = driver.find_element(By.CSS_SELECTOR, ".city-line")
                    city_text = city_tag.text.strip()
                    postal_code = city_text.split()[0]
                    locality_name = " ".join(city_text.split()[1:])
                except:
                    postal_code = None
                    locality_name = None

                # -----------------------
                # Extract price
                # -----------------------
                try:
                    price_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_price_data")
                    price = price_tag.text.strip()
                except:
                    price = None

                # -----------------------
                # Extract property type & subtype
                # -----------------------
                try:
                    type_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_title_main")
                    type_words = type_tag.text.strip().split()
                    property_type = type_words[0]
                except:
                    property_type = None

                # -----------------------
                # Print results
                # -----------------------
                print("Terrace:", terrace)
                print("Garden presence:", garden)
                print("Garden area:", garden_area)
                print("Number of facades:", facades)
                print("Swimming pool:", swimming_pool)
                print("State of building:", state)
                print("Build year:", build_year)
                print("Number of rooms:", number_rooms)
                print("Living area (m²):", living_area)
                print("Equipped kitchen:", equipped_kitchen)
                print("Furnished:", furnished)
                print("Property ID:", property_id)
                print("Postal Code:", postal_code)
                print("Locality:", locality_name)
                print("Price:", price)
                print("Type:", property_type)
                

                # Write row to CSV file
                writer.writerow([url, property_id, property_type, locality_name, postal_code, build_year, state ,furnished, number_rooms,equipped_kitchen, terrace,facades, garden, garden_area,swimming_pool, price])
 
                for row in data:
                    # Replace None with the string "None"
                      writer.writerow([x if x is not None else "None" for x in row.values()])

            except Exception as e:
                print(f"Error loading page: {url}")
                print("Reason:", e)
            finally:
                if driver:
                    driver.quit()




# -----------------------------
# ✅ USE SECTION 
# -----------------------------

input_file = "apartment_urls.csv"
output_file = "properties-1rst.csv"

urls = []

# Read URLs from the CSV (expecting header: Type,URL)
with open(input_file, newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if "URL" in row and row["URL"].startswith("http"):
            urls.append(row["URL"])

print(f"✅ Loaded {len(urls)} URLs from {input_file}")

# Example: call your scraper function (make sure it's defined)
# scrape_urls(urls, output_file)
# print(f"✅ Scraping complete. Results saved in '{output_file}'")


# Run the scraper
scrape_urls(urls, output_file)
print(f"✅ Scraping complete. Results saved in '{output_file}'")

✅ Loaded 20 URLs from apartment_urls.csv
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: 116
Number of facades: 2
Swimming pool: 0
State of building: New
Build year: 2021
Number of rooms: 3
Living area (m²): 170
Equipped kitchen: None
Furnished: 0
Property ID: vbb82041
Postal Code: 4900
Locality: Spa
Price: Make offer from 425 000 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: 2
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 112
Equipped kitchen: None
Furnished: None
Property ID: vbc15133
Postal Code: 4900
Locality: Spa
Price: 350 000 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: 2
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 112
Equipped kitchen: None
Furnished: None
Property ID: vbc15134
Postal Code: 4900
Locality: Spa
Price: 299 000 €
Type: Apartment
Coo

## first 50 pages antwerpen province

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import threading
import csv
import time


# -----------------------
# Handle cookie popups
# -----------------------
def handle_cookies(driver, timeout=10):
    selectors = [
        "#onetrust-accept-btn-handler",
        "#didomi-notice-agree-button",
        "button[aria-label='Accept all']",
    ]
    for selector in selectors:
        try:
            cookie_btn = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
            )
            try:
                cookie_btn.click()
                print(f"Cookies accepted ({selector})")
                return
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].scrollIntoView(true);", cookie_btn)
                time.sleep(0.5)
                cookie_btn.click()
                print(f"Cookies accepted after scrolling ({selector})")
                return
        except TimeoutException:
            continue
    print("No cookie popup found.")


# -----------------------
# Scrape a single province (first N pages)
# -----------------------
def scrape_province(province_slug, province_name, properties_set, lock, max_pages=10):
    options = Options()
    options.add_argument("--headless")  # ✅ correct

    options.add_argument("--width=1920")
    options.add_argument("--height=1080")

    # Path to your GeckoDriver
    gecko_path = r"C:/Users/geckodriver.exe"
    service = Service(gecko_path)
    driver = webdriver.Firefox(service=service, options=options)

    try:
        for page in range(1, max_pages + 1):
            province_url = (
                f"https://immovlan.be/en/real-estate?"
                f"transactiontypes=for-sale&propertytypes=house,apartment&provinces={province_slug}&noindex=1&page={page}"
            )
            driver.get(province_url)
            handle_cookies(driver)
            time.sleep(2)

            print(f"🌍 {province_name} - Page {page}")

            # Scroll down to load all listings
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Wait for property links
            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located(
                        (By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
                    )
                )
            except TimeoutException:
                print(f"No property links found on page {page} in {province_name}")
                continue

            # Extract links
            links = driver.find_elements(By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
            page_urls = {link.get_attribute("href").split("?")[0] for link in links if link.get_attribute("href")}

            # Thread-safe update
            with lock:
                properties_set.update(page_urls)

            print(f"✅ Page {page} - {province_name}: {len(page_urls)} properties found.")

    finally:
        driver.quit()


# -----------------------
# Scrape all provinces
# -----------------------
def scrape_all_provinces(max_pages=10):
    provinces = {
        "Antwerpen": "antwerpen",
        
    }

    properties = set()
    lock = threading.Lock()
    threads = []

    start_time = time.time()

    # Start one thread per province
    for province_name, province_slug in provinces.items():
        t = threading.Thread(
            target=scrape_province,
            args=(province_slug, province_name, properties, lock, max_pages)
        )
        threads.append(t)
        t.start()

    # Wait for all threads to complete
    for t in threads:
        t.join()

    # Save to CSV
    with open("final_urls.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["URL"])
        for url in sorted(properties):
            writer.writerow([url])

    total_properties = len(properties)
    end_time = time.time()
    print(f"\n🏁 Finished scraping in {end_time - start_time:.2f} seconds")
    print(f"Total properties: {total_properties}")
    print("Saved to final_urls.csv")


# -----------------------
# Run
# -----------------------
if __name__ == "__main__":
    scrape_all_provinces(max_pages=10)


Cookies accepted (#didomi-notice-agree-button)
🌍 Antwerpen - Page 1
✅ Page 1 - Antwerpen: 20 properties found.
No cookie popup found.
🌍 Antwerpen - Page 2
✅ Page 2 - Antwerpen: 20 properties found.
No cookie popup found.
🌍 Antwerpen - Page 3
✅ Page 3 - Antwerpen: 20 properties found.
No cookie popup found.
🌍 Antwerpen - Page 4
✅ Page 4 - Antwerpen: 20 properties found.
No cookie popup found.
🌍 Antwerpen - Page 5
✅ Page 5 - Antwerpen: 20 properties found.
No cookie popup found.
🌍 Antwerpen - Page 6
✅ Page 6 - Antwerpen: 20 properties found.
No cookie popup found.
🌍 Antwerpen - Page 7
✅ Page 7 - Antwerpen: 20 properties found.
No cookie popup found.
🌍 Antwerpen - Page 8
✅ Page 8 - Antwerpen: 20 properties found.
No cookie popup found.
🌍 Antwerpen - Page 9
✅ Page 9 - Antwerpen: 20 properties found.
No cookie popup found.
🌍 Antwerpen - Page 10
✅ Page 10 - Antwerpen: 20 properties found.

🏁 Finished scraping in 699.14 seconds
Total properties: 200
Saved to final_urls.csv
